In [1]:
# Even More data when using the query string below and taking out http.method and http.virtual_host

## s = searchContext.query('query_string', query='destination.port:*')

## Empire Beacon is found at 192.168.120.158 on port 443 communicating with 192.168.128.50


In [2]:
#Elasticsearch connector
from elasticsearch_dsl import Search
from elasticsearch import Elasticsearch
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
es = Elasticsearch(['https://192.168.1.181:9200'], timeout=30, max_retries=10, retry_on_timeout=True,                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [4]:
query_since_ts = 0
body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"asc"}}]}'
#body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"desc"}}]}'

In [5]:
#s = searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')

s = searchContext.query('query_string', query='destination.port:*')

#s = searchContext.query('query_string', query='(destination.port:"*" OR http.virtual_host:"*")')

#s = searchContext.query('query_string', query='http.virtual_host:*')

#s = searchContext.query('query_string', query='http.method:*')

#s = searchContext.query('query_string', query='event.module:zeek')


In [6]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [7]:
response = s.execute()
df = pd.DataFrame(columns=['ts','source.ip','destination.ip','destination.port'])
if response.success():
    for d in s[:10000]:
        try:
            df = df.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'destination.port': d['destination']['port']}, ignore_index=True)
        except KeyError:
            pass       

display(df)

df.info()

,ts,source.ip,destination.ip,destination.port
0,2021-12-29T22:25:30.117Z,192.168.128.50,192.168.120.158,443
1,2021-12-29T22:23:29.880Z,192.168.128.50,192.168.120.158,443
2,2021-12-29T22:21:29.647Z,192.168.128.50,192.168.120.158,443
3,2021-12-29T22:19:29.627Z,192.168.128.50,192.168.120.158,443
4,2021-12-29T22:17:29.393Z,192.168.128.50,192.168.120.158,443
...,...,...,...,...
9995,2021-12-29T21:04:42.000Z,192.168.1.85,192.168.1.255,15600
9996,2021-12-29T21:04:42.000Z,192.168.1.85,192.168.1.255,15600
9997,2021-12-29T21:04:42.000Z,192.168.1.85,192.168.1.255,15600
9998,2021-12-29T21:04:42.000Z,192.168.1.85,192.168.1.255,15600


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                10000 non-null  object
 1   source.ip         10000 non-null  object
 2   destination.ip    10000 non-null  object
 3   destination.port  10000 non-null  object
dtypes: object(4)
memory usage: 312.6+ KB


In [8]:
df['ts'] = pd.to_datetime(df['ts'],infer_datetime_format=True)
df.head(5)

,ts,source.ip,destination.ip,destination.port
0,2021-12-29 22:25:30.117000+00:00,192.168.128.50,192.168.120.158,443
1,2021-12-29 22:23:29.880000+00:00,192.168.128.50,192.168.120.158,443
2,2021-12-29 22:21:29.647000+00:00,192.168.128.50,192.168.120.158,443
3,2021-12-29 22:19:29.627000+00:00,192.168.128.50,192.168.120.158,443
4,2021-12-29 22:17:29.393000+00:00,192.168.128.50,192.168.120.158,443


In [9]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_port = 'destination.port'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_port,]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_port,]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_port,'deltas']

In [10]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(50)

ts
source.ip       destination.ip  destination.port                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
0.0.0.0         255.255.255.255 67                [2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2...
0:0:0:0:0:0:0:1 0:0:0:0:0:0:0:1 5985                                                                     [2021-12-29 19:18:53.532000+00:00, 2021-12-29 19:18:53.530000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-30 19:18:52.595000+00:00, 2021-12-30 19:18:52.345000+00:00, 2021-12-30 19:18:51.832000+00:00, 2021-12-30 19:18:51.566000+00:00, 2021-12-30 18:21:18.429000+00:00, 2021-12-30 18:21:18.180000+00:00, 2021-12-30 18:21:17.659000+00:00, 2021-12-30 18:21:17.400000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.020000+00:00, 2021-12-30 15:17:30.019000+00:00]
127.0.0.1       127.0.0.1       53                [2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:34:56+00:00, 2021-12-29 23:34:56+00:00, 2...
146.88.240.4    192.168.1.154   1194                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          [2021-12-29 22:21:47+00:00]
192.168.1.154   104.102.249.16  80                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 [2021-12-29 21:48:59+00:00, 2021-12-29 21:48:49+00:00]
                104.102.249.64  80                                                               

In [11]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,destination.port,ts
0,0.0.0.0,255.255.255.255,67,"[2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2..."
1,0:0:0:0:0:0:0:1,0:0:0:0:0:0:0:1,5985,"[2021-12-29 19:18:53.532000+00:00, 2021-12-29 19:18:53.530000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-30 19:18:52.595000+00:00, 2021-12-30 19:18:52.345000+00:00, 2021-12-30 19:18:51.832000+00:00, 2021-12-30 19:18:51.566000+00:00, 2021-12-30 18:21:18.429000+00:00, 2021-12-30 18:21:18.180000+00:00, 2021-12-30 18:21:17.659000+00:00, 2021-12-30 18:21:17.400000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.020000+00:00, 2021-12-30 15:17:30.019000+00:00]"
2,127.0.0.1,127.0.0.1,53,"[2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:59:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:54:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:49:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:44:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:39:56+00:00, 2021-12-29 23:34:56+00:00, 2021-12-29 23:34:56+00:00, 2..."
3,146.88.240.4,192.168.1.154,1194,[2021-12-29 22:21:47+00:00]
4,192.168.1.154,104.102.249.16,80,"[2021-12-29 21:48:59+00:00, 2021-12-29 21:48:49+00:00]"


In [12]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count
0,0.0.0.0,255.255.255.255,67,"[2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:22+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 23:18:20+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:44+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2021-12-29 22:52:43+00:00, 2...",68
1,0:0:0:0:0:0:0:1,0:0:0:0:0:0:0:1,5985,"[2021-12-29 19:18:53.532000+00:00, 2021-12-29 19:18:53.530000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-29 19:18:52.771000+00:00, 2021-12-30 19:18:52.595000+00:00, 2021-12-30 19:18:52.345000+00:00, 2021-12-30 19:18:51.832000+00:00, 2021-12-30 19:18:51.566000+00:00, 2021-12-30 18:21:18.429000+00:00, 2021-12-30 18:21:18.180000+00:00, 2021-12-30 18:21:17.659000+00:00, 2021-12-30 18:21:17.400000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.271000+00:00, 2021-12-30 15:17:30.020000+00:00, 2021-12-30 15:17:30.019000+00:00]",16


In [19]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 50]
df.shape

(8, 18)

In [20]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:03:30.945000+00:00, 2021-12-29 19:05:31.420000+00:00, 2021-12-29 19:07:31.521000+00:00, 2021-12-29 19:09:31.822000+00:00, 2021-12-29 19:11:31.943000+00:00, 2021-12-29 19:13:32.214000+00:00, 2021-12-29 19:15:32.334000+00:00, 2021-12-29 19:17:32.665000+00:00, 2021-12-29 19:19:32.838000+00:00, 2021-12-29 19:21:32.975000+00:00, 2021-12-29 19:23:33.380000+00:00, 2021-12-29 19:25:33.518000+00:00, 2021-12-29 19:27:33.919000+00:00, 2021-12-29 19:29:33.990000+00:00, 2021-12-29 19:31:34.419000+00:00, 2021-12-29 19:33:34.643000+00:00, 2021-12-29 19:35:34.882000+00:00, 2021-12-29 19:37:...",167,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ...",120.0,120.0,120.0,0.0,0.0,0.0,0.0,146.6,1.0,1.0,1.000000,1.000000
1,192.168.1.85,192.168.1.255,15600,"[2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2...",8216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,914.3,1.0,1.0,1.000000,1.000000
2,0.0.0.0,255.255.255.255,67,"[2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2...",68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 908.0, 0.0, 0.0, 0.0, 294.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1536.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.2,1.0,1.0,0.247093,0.749031
3,127.0.0.1,127.0.0.1,53,"[2021-12-29 21:04:56+00:00, 2021-12-29 21:04:56+00:00, 2021-12-29 21:04:56+00:00, 2021-12-29 21:04:56+00:00, 2021-12-29 21:09:56+00:00, 2021-12-29 21:09:56+00:00, 2021-12-29 21:09:56+00:00, 2021-12-29 21:09:56+00:00, 2021-12-29 21:14:

In [21]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:03:30.945000+00:00, 2021-12-29 19:05:31.420000+00:00, 2021-12-29 19:07:31.521000+00:00, 2021-12-29 19:09:31.822000+00:00, 2021-12-29 19:11:31.943000+00:00, 2021-12-29 19:13:32.214000+00:00, 2021-12-29 19:15:32.334000+00:00, 2021-12-29 19:17:32.665000+00:00, 2021-12-29 19:19:32.838000+00:00, 2021-12-29 19:21:32.975000+00:00, 2021-12-29 19:23:33.380000+00:00, 2021-12-29 19:25:33.518000+00:00, 2021-12-29 19:27:33.919000+00:00, 2021-12-29 19:29:33.990000+00:00, 2021-12-29 19:31:34.419000+00:00, 2021-12-29 19:33:34.643000+00:00, 2021-12-29 19:35:34.882000+00:00, 2021-12-29 19:37:...",167,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ...",120.0,120.0,120.0,0.0,0.0,0.0,0.0,146.6,1.0,1.0,1.0,1.0
1,192.168.1.85,192.168.1.255,15600,"[2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2...",8216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,914.3,1.0,1.0,1.0,1.0


In [22]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 10)

In [23]:
df.head(3)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:03:30.945000+00:00, 2021-12-29 19:05:31.420000+00:00, 2021-12-29 19:07:31.521000+00:00, 2021-12-29 19:09:31.822000+00:00, 2021-12-29 19:11:31.943000+00:00, 2021-12-29 19:13:32.214000+00:00, 2021-12-29 19:15:32.334000+00:00, 2021-12-29 19:17:32.665000+00:00, 2021-12-29 19:19:32.838000+00:00, 2021-12-29 19:21:32.975000+00:00, 2021-12-29 19:23:33.380000+00:00, 2021-12-29 19:25:33.518000+00:00, 2021-12-29 19:27:33.919000+00:00, 2021-12-29 19:29:33.990000+00:00, 2021-12-29 19:31:34.419000+00:00, 2021-12-29 19:33:34.643000+00:00, 2021-12-29 19:35:34.882000+00:00, 2021-12-29 19:37:...",167,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ...",120.0,120.0,120.0,0.0,0.0,0.0,0.0,146.6,1.0,1.0,1.000000,1.000000
1,192.168.1.85,192.168.1.255,15600,"[2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:42+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:48+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2021-12-29 21:04:54+00:00, 2...",8216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,914.3,1.0,1.0,1.000000,1.000000
2,0.0.0.0,255.255.255.255,67,"[2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:30+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:35+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2021-12-29 22:32:36+00:00, 2...",68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 908.0, 0.0, 0.0, 0.0, 294.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1536.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,275.2,1.0,1.0,0.247093,0.749031


In [24]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,167,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ..."
1,1.000000,8216,192.168.1.85,192.168.1.255,15600,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, ...]"
2,0.749031,68,0.0.0.0,255.255.255.255,67,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 908.0, 0.0, 0.0, 0.0, 294.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1536.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]"
3,0.712381,144,127.0.0.1,127.0.0.1,53,"[0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 1.0, 0.0, 299.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 300.0, ...]"
4,0.706333,120,192.168.1.220,192.168.1.255,138,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 722.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 717.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 718.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 721.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 718.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 723.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 722.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 719.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 720.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 722.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 720.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 719.0, 0.0, 0.0, 0.0, 0.0, ...]"
5,0.686498,60,192.168.1.91,192.168.1.255,138,"[0.0, 0.0, 0.0, 722.0, 0.0, 0.0, 0.0, 723.0, 0.0, 0.0, 0.0, 721.0, 0.0, 0.0, 0.0, 717.0, 0.0, 0.0, 0.0, 717.0, 0.0, 0.0, 0.0, 721.0, 0.0, 0.0, 0.0, 720.0, 0.0, 0.0, 0.0, 721.0, 0.0, 0.0, 0.0, 719.0, 0.0, 0.0, 0.0, 722.0, 0.0, 0.0, 0.0, 723.0, 0.0, 0.0, 0.0, 719.0, 0.0, 0.0, 0.0, 721.0, 0.0, 0.0, 0.0, 719.0, 0.0, 0.0, 0.0]"
6,0.290437,119,192.168.1.154,13.107.222.240,53,"[268.0, 61.0, 135.0, 4.0, 3.0, 126.0, 102.0, 1.0, 60.0, 349.0, 159.0, 53.0, 71.0, 60.0, 116.0, 245.0, 130.0, 16.0, 44.0, 131.0, 15.0, 0.0, 115.0, 201.0, 19.0, 50.0, 192.0, 183.0, 112.0, 1.0, 170.0, 85.0, 1.0, 15.0, 91.0, 15.0, 201.0, 9.0, 76.0, 3.0, 0.0, 7.0, 8.0, 2.0, 5.0, 0.0, 158.0, 66.0, 381.0, 2.0, 37.0, 248.0, 107.0, 60.0, 3.0, 1.0, 219.0, 23.0, 385.0, 73.0, 72.0, 4.0, 119.0, 44.0, 60.0, 92.0, 91.0, 0.0, 13.0, 1.0, 8.0, 1.0, 1.0, 4.0, 102.0, 241.0, 100.0, 20.0, 40.0, 10.0, 327.0, 26.0, 44.0, 5.0, 87.0, 215.0, 143.0, 17.0, 304.0, 462.0, 74.0, 2.0, 0.0, 0.0, 2.0, 56.0, 49.0, 183.0, 100..."
7,0.271469,125,192.168.1.154,192.168.1.181,5044,"[59.0, 210.0, 147.0, 64.0, 179.0, 78.0, 15.0, 88.0, 129.0, 26.0, 25.0, 89.0, 2.0, 126.0, 234

In [ ]:
df.loc[df['tsScore'] > 0.80, columns_to_display]